In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
statisticCategory = input()

In [ ]:
# Datset could have multiple pages.  We will set on page 1 to begin
p = 1

In [ ]:
payload = {"statisticCategory":statisticCategory.upper(),"seasonType":"REG","d-447263-p":str(p)}
url = 'http://www.nfl.com/stats/categorystats'
response = requests.get(url,params=payload)


In [ ]:
print("Response:", response.status_code,response.url) #200 means it went through


In [ ]:
soup = BeautifulSoup(response.text,'html.parser')

In [ ]:
#The website table is partitioned into multiple pages.  We need to know how many pages to iterate

pagingText = soup.find("span", {"class": "linkNavigation floatRight"})

pagingLinks = pagingText.findAll("a")
pages = len(pagingLinks)
print("Number of page links:",len(pagingLinks))

In [ ]:
#Get Header for the table
columnHeader = []
header = soup.find('table', attrs={'id':'result'}).find_all('th')
for c in header:
    columnHeader.append(c.text)
columnHeader = [c.strip('\n') for c in columnHeader]


In [ ]:
tableRows = []

for pg in range(1,pages+1):
    
    #pause our code for a second so that we are not spamming the website with requests. This helps us avoid getting flagged as a spammer.
    time.sleep(1)
    
    #update page number in payload
    payload["d-447263-p"] = pg
    response = requests.get(url,params=payload)
    print("Page:",pg,"url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table', attrs={'id':'result'})
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    
   


In [ ]:
resultsDF = pd.DataFrame(tableRows, columns=columnHeader)
display(resultsDF.head(),resultsDF.shape)


In [ ]:
#Save results to Excel
baseFileName = "NFL_Stats_Demo_"

resultsDF.to_excel(baseFileName + statisticCategory + ".xlsx")